## Benchmarking task - Q-A/RAG

#### Install python packages requested by benchmarking

If you have not install the requested python libraries, uncomment the following command to run the installation.

In [ ]:
#!pip install -r ../peccyben/requirements.txt

#### Import libraries

In [ ]:
import pandas as pd
import sys

sys.path.insert(0, '../')

from peccyben.qaragtask import QA_Ben, QA_Create_VDB
from peccyben.utils import Ben_Save
from peccyben.promptcatalog import Prompt_Template_Gen

#### Configuration

Setup your environment parameters 

* **BENCH_KEY**: a unique keyname for your benchmarking in this round 
* **S3_BUCKET**: the S3 buckt you created for the benchmarking    
* **TASK_FOLDER**: the task folder you created under the S3 bucket   
* **INPUT_FILE**: the file name of the dataset you prepared for benchmarking
* **DOC_FILE**: the document to query 
* **VDB_NAME**: the vector db name for RAG pipeline   
* **METRICS_LIST**: the metrics we provide for the question-answer task    
* **BEDROCK_REGION**: the AWS region that the model benchmarking runs on Bedrock
* **COST_FILE**: the price file used for calculating model inference cost      

In [ ]:
BENCH_KEY = 'PeccyBen_202503_test'
S3_BUCKET = 'genai-sdo-llm-ben-20240310'
TASK_FOLDER = 'ben-qa'
INPUT_FILE = 'cuad_test_50p.csv'
DOC_FILE = 'PACIRA_PHARMACEUTICALS_AGREEMENT.PDF'
VDB_NAME = "loha"
METRICS_LIST = ['Inference_Time','Input_Token','Output_Token','Throughput','RougeL-sum','Semantic_Similarity','BERT-F1','Toxicity',
                                            'Answer_Correctness','Answer_similarity','Answer_Relevancy','Context_Recall','Context_Precision','Cost','Cache_Input_Token','Cache_Output_Token']
BEDROCK_REGION = 'us-east-1'
COST_FILE = 'bedrock_od_public.csv'

Results_qa = pd.DataFrame()
Results_qa = Results_qa.assign(metric_name=METRICS_LIST) 
Results_qa = Results_qa.set_index('metric_name')

#### Create a vector DB for RAG pipeline

You can skip this step if you already created a vector db in the previous round of benchmarking

* Embedding model: here we use Titan text embedding model on Bedrock
* Set the chunk size and overlap size for your retriever embedding
* Invoke **QA_Create_VDB** to create a faiss vector store

In [ ]:
import boto3
import botocore
from langchain.embeddings import BedrockEmbeddings

boto3_bedrock = boto3.client(service_name="bedrock", region_name="us-east-1")
boto3_bedrock_runtime = boto3.client(service_name="bedrock-runtime", region_name="us-east-1")

# Create embedding
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0", client=boto3_bedrock_runtime)

In [ ]:
# Set chunk size and create the vector db
chunk_size = 1000
chunk_overlap = 100

QA_Create_VDB(chunk_size,chunk_overlap,bedrock_embeddings,VDB_NAME,S3_BUCKET,DOC_FILE,TASK_FOLDER)

#### Task specific setting

* Configure your **prompt** in the prompt catalog (prompt_catalog.json), and configure the prompt_catalog_id
* Set the **LLM hyperparameter** in model_kwargs. For the models on Bedrock, refer to [inferenceConfig](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference-call.html)
* Set **LLM-judge model** for RAGAS 

In [ ]:
prompt_catalog_id = "qa-1"

model_kwargs = {
        'maxTokens': 1024, 
        'topP': 0.9, 
        'temperature': 0
}   
judge_model_id = "mistral.mistral-large-2402-v1:0"

#### Specify the model and other settings for benchmarking

Invoke **QA_Ben** function to conduct the benchmarking for one selected model, repeat for multiple models you want to benchmark

* **method**: set "Bedrock" for the models on Bedrock
* **region**: configured in the previous step
* **model_id**: specify the Model ID for the model endpoint
* **judge_model_id**: specify LLM-judge model for RAGAS
* **model_kwargs**: configured in previous step
* **prompt_template**: prompt template based on the prompt configured in previous step
* **s3_bucket**: configured in previous step
* **file_name**: configured in previous step
* **BENCH_KEY**: configured in previous step
* **task_folder**: configured in previous step
* **cost_key**: set "public" when using AWS public pricing to calculate the cost
* **save_id**: the model name displayed in the report 
* **SLEEP_SEC**: you can configure "sleep and retry" when throtting, for example, set SLEEP_SEC = 10 to wait for 10 seconds between each inference
* **SAMPLE_LEN**: you can configure the number of samples for inference
* **PP_TIME**: if you want to run model inference for multiple rounds, set the number of rounds here.  
* **cacheconf**: set "default" to enable Bedrock Prompt Caching in the inference, "None" to disable
* **latencyOpt**: set "optimized" to enable Bedrock Latency Optimized Inference, "None" to disable

In [ ]:
# nova-pro 
model_id = 'us.amazon.nova-pro-v1:0' 
save_id = 'nova-pro'

prompt_template = Prompt_Template_Gen(model_id, prompt_catalog_id)
#print(prompt_template)

Results_qa[save_id] = QA_Ben(method="Bedrock",
                                   region=BEDROCK_REGION,
                                   model_id=model_id,
                                   judge_model_id=judge_model_id,
                                   model_kwargs=model_kwargs,
                                   prompt_template=prompt_template,
                                   vdb_name=VDB_NAME,
                                   s3_bucket=S3_BUCKET,
                                   file_name=INPUT_FILE,
                                   BENCH_KEY=BENCH_KEY,
                                   task_folder=TASK_FOLDER,
                                   cost_key=COST_FILE,
                                   save_id=save_id,
                                   SLEEP_SEC=20,SAMPLE_LEN=3,
                                   PP_TIME=1,
                                   cacheconf="None",latencyOpt="None")
Results_qa

In [ ]:
# Claude 3.5v2 Sonnet
model_id = 'us.anthropic.claude-3-5-sonnet-20241022-v2:0' 
save_id = 'Sonnet-3.5v2'

prompt_template = Prompt_Template_Gen(model_id, prompt_catalog_id)
#print(prompt_template)

Results_qa[save_id] = QA_Ben(method="Bedrock",
                                   region=BEDROCK_REGION,
                                   model_id=model_id,
                                   judge_model_id=judge_model_id,                             
                                   model_kwargs=model_kwargs,
                                   prompt_template=prompt_template,
                                   vdb_name=VDB_NAME,
                                   s3_bucket=S3_BUCKET,
                                   file_name=INPUT_FILE,
                                   BENCH_KEY=BENCH_KEY,
                                   task_folder=TASK_FOLDER,
                                   cost_key=COST_FILE,
                                   save_id=save_id,
                                   SLEEP_SEC=20,SAMPLE_LEN=3,
                                   PP_TIME=1,
                                   cacheconf="None",latencyOpt="None")
Results_qa

#### Generate benchmarking report

Invoke **Ben_Save** function to generate your benchmarking report and cost-performance analysis, all the results are stored in S3 bucket

* **Results_qa**: the benchmarking results generated in previous step
* **S3_BUCKET**: configured in previous step
* **BENCH_KEY**: configured in previous step
* **TASK_FOLDER**: configured in previous step
* **perf_metric**: select the performance metric from the metrics list in the previous step, for cost-performance analysis, for example, to analyze the accuracy by semantic similarity score with cost, set "Semantic_Similarity".  
* **top_x**: set the top x number of models you want to run the cost-performance analysis 
* **TITLE**: specify a title for the reports and charts

In [ ]:
perf_metric = 'Semantic_Similarity'
top_x = 5

Ben_Save(Results_qa,S3_BUCKET,BENCH_KEY,TASK_FOLDER,perf_metric,top_x,'Q-A/RAG-Task')